# Automatic-Data-Cleaning# Automatic Data Cleaning & Analysis Agent

## 🎯 Objective

This project implements an **Automatic Data Cleaning & Analysis Agent** that:
- Loads raw **CSV/Excel** datasets
- Automatically **profiles**, **cleans**, and **analyzes** the data
- Exports:
  - A **cleaned dataset** ready for modeling

## 🧩 Problem

Data scientists and analysts spend a lot of time:
- Inspecting dataset structure (shape, columns, dtypes)
- Handling missing values and duplicates
- Fixing inconsistent types
- Detecting simple outliers
- Writing the same boilerplate code again and again

This preprocessing phase is:
- Time-consuming
- Error-prone
- Repeated in every new project

## ✅ Proposed Solution

Build an **agent-style system** that:

1. **Loads** any CSV/Excel file
2. **Analyzes** the structure:
   - Shape, columns, data types
   - Missing value report
   - Duplicates
3. **Cleans** the data:
   - Handles missing values (per column type)
   - Detects & removes duplicates
   - Fixes inconsistent types
   - Detects basic outliers

In [1]:
import os
import pandas as pd
import asyncio
import tempfile
from google.adk.agents import LlmAgent
from google.adk.tools.mcp_tool import McpToolset
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters # Required for defining server parameters
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types


In [ ]:
import nest_asyncio
nest_asyncio.apply()
print("✅ Applied nest-asyncio patch for Notebook environment.")

✅ Applied nest-asyncio patch for Notebook environment.


In [13]:
TEMP_DIR_ABS = "C:\\MCP_Test"  # Ensure this directory exists!
TEMP_DIR = pathlib.Path(TEMP_DIR_ABS)
FILE_NAME = "data_to_clean.csv"
TARGET_FILE_PATH = TEMP_DIR / FILE_NAME
# ----------------------------------------

In [2]:
import nest_asyncio
nest_asyncio.apply()


### --- Configuration & Setup ---

#### 1. Setup the Target Directory and File
#### We use tempfile to ensure we get an ABSOLUTE path that exists and is accessible.

In [ ]:

GOOGLE_API_KEY = "*****" 
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY 

APP_NAME = "mcp_file_agent"
USER_ID = "mcp_user"
MODEL_NAME = "gemini-2.5-flash-lite" 

TEMP_DIR = tempfile.mkdtemp()
FILE_NAME = "data_to_clean.csv"
TARGET_FILE_PATH = os.path.join(TEMP_DIR, FILE_NAME)

df_test = pd.DataFrame({
    "col_a": [1, 2, None, 4, None, 5, 1000],
    "col_b": ["cat", "cat", None, "dog", "dog", None, "dog"],
    "col_c": [None, 10, 12, None, 14, None, 16]
})
df_test.to_csv(TARGET_FILE_PATH, index=False)

print(f"💾 Local file created at: {TARGET_FILE_PATH}")
print(f"📁 MCP server will access directory: {TEMP_DIR}")


💾 Local file created at: C:\Users\dell\AppData\Local\Temp\tmpmol3_5hi\data_to_clean.csv
📁 MCP server will access directory: C:\Users\dell\AppData\Local\Temp\tmpmol3_5hi



### --- 2. Tool Definition (Placeholder for your cleaning tool) ---
### Since the LLM will first use the MCP tool (read_file) to get the data,
### we need a placeholder tool to receive the content.



In [ ]:
def clean_csv_data(csv_content: str) -> str:
    """Nettoie les données CSV en supprimant les lignes avec des valeurs manquantes"""
    try:
        # Extraire juste le CSV si le contenu contient du texte avant
        lines = csv_content.strip().split('\n')
        csv_start = 0
        for i, line in enumerate(lines):
            if ',' in line and any(c.isalpha() for c in line):
                csv_start = i
                break
        
        csv_only = '\n'.join(lines[csv_start:])
        
        # Charger et nettoyer
        df = pd.read_csv(StringIO(csv_only))
        rows_before = len(df)
        
        # Nettoyage : supprimer lignes avec NaN
        df_cleaned = df.dropna()
        rows_after = len(df_cleaned)
        rows_removed = rows_before - rows_after
        
        # Résumé
        summary = f"""📊 Résumé du nettoyage :
- Lignes initiales : {rows_before}
- Lignes supprimées (avec NaN) : {rows_removed}
- Lignes finales : {rows_after}
- Colonnes : {list(df_cleaned.columns)}

🧹 Données nettoyées :
```csv
{df_cleaned.to_csv(index=False)}```

💾 Pour sauvegarder : df_cleaned.to_csv('data_cleaned.csv', index=False)
"""
        
        # Optionnel : sauvegarder automatiquement
        cleaned_path = DATA_DIR / "data_cleaned.csv"
        df_cleaned.to_csv(cleaned_path, index=False)
        summary += f"\n✅ Fichier nettoyé sauvegardé : {cleaned_path}"
        
        return summary
        
    except Exception as e:
        return f"❌ Erreur lors du nettoyage : {e}"



## --- 3. Agent and MCP Toolset Configuration ---


In [ ]:

# Instantiate the McpToolset, pointing it to the Filesystem Server
filesystem_toolset = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command='npx',
            args=[
                "-y",  # Auto-confirm install
                "@modelcontextprotocol/server-filesystem",
                os.path.abspath(TEMP_DIR), # IMPORTANT: Pass the ABSOLUTE path
            ],
        ),
    ),
    tool_filter=['list_directory', 'read_file'] # Only expose necessary file tools
)

root_agent = LlmAgent(
    model=MODEL_NAME,
    name=APP_NAME,
    instruction=f"""You are a helpful assistant specialized in data cleaning. 
    1. You have tools to manage files using the Model Context Protocol (MCP).
    2. The target file for cleaning is always named '{FILE_NAME}'.
    3. **First, you MUST use the 'read_file' MCP tool** to read the content of '{FILE_NAME}'.
    4. **Second, you MUST pass the resulting file content (the raw CSV string) to the 'process_csv_content_tool'** for cleaning and final output.
    5. Do NOT perform cleaning yourself. State the result provided by the 'process_csv_content_tool' after the tool chain is complete.
    """,
    tools=[
        filesystem_toolset,
        clean_csv_data
    ]
)

session_service = InMemorySessionService()
runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)

print("✅ MCP-enabled agent initialized!")


✅ MCP-enabled agent initialized!


### --- 4. Helper Function for Async Execution ---

In [ ]:



# Fix for the "RuntimeError: asyncio.run() cannot be called from a running event loop"
async def run_session(
    runner_instance: Runner,
    prompt: str,
    session_name: str = "default",
):
    print(f"\n### Session: {session_name}")

    try:
        session = await runner_instance.session_service.create_session(
            app_name=runner_instance.app_name, user_id=USER_ID, session_id=session_name
        )
    except Exception:
        session = await runner_instance.session_service.get_session(
            app_name=runner_instance.app_name, user_id=USER_ID, session_id=session_name
        )

    msg = types.Content(role="user", parts=[types.Part(text=prompt)])
    
    print(f"User > {prompt}")

    async for event in runner_instance.run_async(
        user_id=USER_ID, session_id=session.id, new_message=msg
    ):
        if event.content and event.content.parts:
            text = event.content.parts[0].text
            if text and text != "None":
                print(f"{MODEL_NAME} > ", text)
            
            # Print function calls/responses for transparency
            for part in event.content.parts:
                if part.function_call:
                    print(f"Tool Call: {part.function_call.name}({dict(part.function_call.args)})")
                if part.function_response:
                    print(f"Tool Response: {part.function_response.name} returned {part.function_response.response}")



In [ ]:
prompt = f"Please clean the local file '{FILE_NAME}' now."
await run_session(runner, prompt, "my-data-cleaning-session")